In [2]:
import pandas as pd
import re

In [3]:
#changes float views from scientific
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
#load April - July, 2017 & 2018 MTA Turntile data
#time intensive block
df = pd.read_csv('/Users/GabeKlick/Downloads/turnstile_recordings_summer_2017_2018_clean.csv', index_col = 'Unnamed: 0')
df.head()

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [5]:
#exit column has a few misplaced chars preventing the column from being int values
#this block removes the chars and converts columns to int64

#apply regex statement to ensure only numbers in the column
def only_num(col):
    return re.findall('(\d)', col)
df.exits = df.exits.apply(lambda x: only_num(str(x)))

#the regex statment returns a list a numbers for each value
#join list values to single string items of numbers
df.exits = df.exits.apply(lambda x: ''.join(x))

#convert str to int64
df.exits = df.exits.apply(lambda x: int(x))

df.head()

,c_a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/29/2017,16:00:00,REGULAR,6158212,2085529


In [6]:
#dropped df.date conversion - which was a speparate from the date_time combine conversion

#combine and convert date,time column from str to datetime
#time intensive block, run then take 5

df.time = pd.to_datetime(df.date + ' ' + df.time)

#rename to showcase the new time feature in the column
df.rename({'time': 'date_time'}, axis = 'columns', inplace = True)

#removes the now date time column
df.drop(columns = ['date'], inplace = True)

df.head()

,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits
0,"[""A002",R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 00:00:00,REGULAR,6157740,2085315
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 04:00:00,REGULAR,6157777,2085319
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 08:00:00,REGULAR,6157810,2085353
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 12:00:00,REGULAR,6157963,2085453
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-04-29 16:00:00,REGULAR,6158212,2085529


In [7]:
#ensure that date_time is increasing in chronical order
#this is helpful later when grouping by turnstiles and station
df = df.sort_values(by = 'date_time', ascending = True)

In [8]:
#create pickle for quicker processing time
df.to_pickle('date_formatted_df.pkl')

In [9]:
df = pd.read_pickle('date_formatted_df.pkl')

In [10]:
# remove dates before May and after August
df = df[(df.date_time.dt.month != 4) & (df.date_time.dt.month != 8)].reset_index()
df.head()

,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits
0,47893,N057,R188,00-05-00,50 ST,CE,IND,2017-05-01,REGULAR,2902,0
1,29651,H008,R248,01-00-01,1 AV,L,BMT,2017-05-01,REGULAR,2252969,12850185
2,43798,N025,R102,01-00-03,125 ST,ACBD,IND,2017-05-01,REGULAR,2492387,1976011
3,38158,J031,R006,00-00-02,WOODHAVEN BLVD,JZ,BMT,2017-05-01,REGULAR,3835875,1958721
4,41428,N010,R126,00-03-02,175 ST,A,IND,2017-05-01,REGULAR,2309478,2471918


In [11]:
#replace typo in c_a column
df.loc[df['c_a'] == ' "A002', 'c_a'] = 'A002'

In [12]:
# grouping by turnstile id (station, unit, c_a, scp), find the difference between consecutive data points
# for each individual turnstile. This method is applied to both the exit, entry, and date_time for each observation
entries_difference_series = df.groupby(['station', 'unit', 'c_a', 'scp'])['entries'].apply(lambda x: x - x.shift(1))
exits_difference_series = df.groupby(['station', 'unit', 'c_a', 'scp'])['exits'].apply(lambda x: x - x.shift(1))
time_elapsed_series = df.groupby(['station', 'unit', 'c_a', 'scp'])['date_time'].apply(lambda x: x - x.shift(1))

#these 3 series are ardded to a temp data frame that will be merged to the main dataframe
temp_df = pd.DataFrame({'Index': df.index, 'num_entries': entries_difference_series, 'num_exits': exits_difference_series, 'time_elapsed': time_elapsed_series})

In [13]:
#show temp df
temp_df.head()

,Index,num_entries,num_exits,time_elapsed
0,0,nan,nan,NaT
1,1,nan,nan,NaT
2,2,nan,nan,NaT
3,3,nan,nan,NaT
4,4,nan,nan,NaT


In [14]:
# creating a new dataframe for grouping at the turnstile level
# time intensive, run then take 5
# merge the main and temp df on the index column
turnstile_df = pd.concat([df, temp_df], axis = 1, join = 'outer')
turnstile_df.head()

,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,time_elapsed
0,47893,N057,R188,00-05-00,50 ST,CE,IND,2017-05-01,REGULAR,2902,0,0,nan,nan,NaT
1,29651,H008,R248,01-00-01,1 AV,L,BMT,2017-05-01,REGULAR,2252969,12850185,1,nan,nan,NaT
2,43798,N025,R102,01-00-03,125 ST,ACBD,IND,2017-05-01,REGULAR,2492387,1976011,2,nan,nan,NaT
3,38158,J031,R006,00-00-02,WOODHAVEN BLVD,JZ,BMT,2017-05-01,REGULAR,3835875,1958721,3,nan,nan,NaT
4,41428,N010,R126,00-03-02,175 ST,A,IND,2017-05-01,REGULAR,2309478,2471918,4,nan,nan,NaT


In [15]:
# removed all rows where the time elapsed was greater than 60 days
#This drops a negligible amount of date (< .01%)
turnstile_df = turnstile_df[turnstile_df['time_elapsed'].dt.days <= 60]

In [16]:
# removed rows with negative time elapses.
# needs to be greated than OR equal to because all time elapses within 24 hours are 0 or dt.days
#This drops a negligible amount of date (< .01%)
turnstile_df = turnstile_df[turnstile_df['time_elapsed'].dt.days >= 0]

In [17]:
'''
Create a variable that tracks the rate of passengers for each turnstile per hour

Numerator: add the num enteries/exit per observation that was found before the merge.
This gives total foot traffic per turnstile per observation

Denominator: time elapsed provides the difference between each observation for each turnstile.
This observation can be pulled by full days (year,mont, day converted to day) 
and partial day (hour, min, seconds to seconds).
For each observation the full and partial day info is converted to seconds (24hours*60min*60sec = 86400seconds)
then converted back to hour format (60min*60seconds = 3600 seconds)''

'''
turnstile_df['passengers_per_hour_per_turnstile'] = (turnstile_df.num_entries + turnstile_df.num_exits) \
/(((turnstile_df.time_elapsed.dt.days * 86400) + (turnstile_df.time_elapsed.dt.seconds))/3600)

In [18]:
# remove outliers for passengers per hour per turnstile that are greater than 1000
# this removes .01% of the data
turnstile_df = turnstile_df[turnstile_df['passengers_per_hour_per_turnstile'] < 1000]

In [19]:
# removed all rows where the num_entries OR the num_exits was less than or equal to 0
# logically, turnstiles should alway be increasing throughout time. However, due to errors in the systems
# such as resets there can be cases of negative values

# This removes 16.65% of the data
turnstile_df = turnstile_df[(turnstile_df['num_entries'] > 0) & (turnstile_df['num_exits'] > 0)]

In [20]:
# combine total passengers for each turnstile reporting
turnstile_df['total_passengers'] = turnstile_df.num_entries + turnstile_df.num_exits
turnstile_df.head()

,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,time_elapsed,passengers_per_hour_per_turnstile,total_passengers
1804,17094,B024A,R211,02-00-03,KINGS HWY,BQ,BMT,2017-05-01 04:00:00,REGULAR,300189,413283,1804,4.000,34.000,04:00:00,9.500,38.000
1805,11337,A069,R044,01-06-00,CHAMBERS ST,JZ456,BMT,2017-05-01 04:00:00,REGULAR,2153424,18150488,1805,3.000,25.000,04:00:00,7.000,28.000
1806,56635,N094,R029,01-00-02,WORLD TRADE CTR,ACE23,IND,2017-05-01 04:00:00,REGULAR,4904359,3433675,1806,12.000,14.000,04:00:00,6.500,26.000
1807,75211,N305A,R016,00-03-04,LEXINGTON AV/53,EM6,IND,2017-05-01 04:00:00,REGULAR,10541078,13601575,1807,9.000,8.000,04:00:00,4.250,17.000
1809,26002,E009,R370,00-00-00,71 ST,D,BMT,2017-05-01 04:00:00,REGULAR,7175937,7106382,1809,9.000,25.000,04:00:00,8.500,34.000


In [21]:
'''time elapsed provides the difference between each observation for each turnstile.
This observation can be pulled by full days (year,mont, day converted to day) 
and partial day (hour, min, seconds to seconds).
For each observation the full and partial day info is converted to seconds (24hours*60min*60sec = 86400seconds)
then converted back to hour format (60min*60seconds = 3600 seconds) '''

#create column tracking total hours passed for each turnstile reporting
turnstile_df['hours_elapsed'] = (((turnstile_df['time_elapsed'].dt.days * 86400) + (turnstile_df['time_elapsed'].dt.seconds)))/3600
turnstile_df.head()

,index,c_a,unit,scp,station,linename,division,date_time,desc,entries,exits,Index,num_entries,num_exits,time_elapsed,passengers_per_hour_per_turnstile,total_passengers,hours_elapsed
1804,17094,B024A,R211,02-00-03,KINGS HWY,BQ,BMT,2017-05-01 04:00:00,REGULAR,300189,413283,1804,4.000,34.000,04:00:00,9.500,38.000,4.000
1805,11337,A069,R044,01-06-00,CHAMBERS ST,JZ456,BMT,2017-05-01 04:00:00,REGULAR,2153424,18150488,1805,3.000,25.000,04:00:00,7.000,28.000,4.000
1806,56635,N094,R029,01-00-02,WORLD TRADE CTR,ACE23,IND,2017-05-01 04:00:00,REGULAR,4904359,3433675,1806,12.000,14.000,04:00:00,6.500,26.000,4.000
1807,75211,N305A,R016,00-03-04,LEXINGTON AV/53,EM6,IND,2017-05-01 04:00:00,REGULAR,10541078,13601575,1807,9.000,8.000,04:00:00,4.250,17.000,4.000
1809,26002,E009,R370,00-00-00,71 ST,D,BMT,2017-05-01 04:00:00,REGULAR,7175937,7106382,1809,9.000,25.000,04:00:00,8.500,34.000,4.000


In [22]:
#pickle the data frame
turnstile_df.to_pickle('turnstile_df.pkl')

In [23]:
#load pickle
turnstile_df = pd.read_pickle('turnstile_df.pkl')

In [24]:
# aggregations for new dataframe 'see below'
#grouping by station and time segment,
#sum across turnstiles in stations
#sum the hours elapsed for each grouping
#count the number of turnstiles in each grouping
#the sum of hours elapsed and turnstiles count should logically divide to the time elapsed
aggregations = {
    'total_passengers': 'sum',
    'hours_elapsed': 'sum',
    'num_entries': 'count'
}

In [25]:
# create a new dataframe by station by date
#apply aggregations from above
date_df = turnstile_df.groupby(('station','date_time')).agg(aggregations).reset_index()
date_df.rename({'hours_elapsed': 'turnstile_hours', 'num_entries': 'num_turnstiles'}, axis = 1, inplace = True)

/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
date_df.head()

,station,date_time,total_passengers,turnstile_hours,num_turnstiles
0,1 AV,2017-05-01 04:00:00,690.000,36.000,9
1,1 AV,2017-05-01 08:00:00,5100.000,40.000,10
2,1 AV,2017-05-01 12:00:00,10538.000,40.000,10
3,1 AV,2017-05-01 16:00:00,8687.000,40.000,10
4,1 AV,2017-05-01 20:00:00,12671.000,40.000,10


In [27]:
#passengers per hour is total passengers per station (summed turnstiles totals)

#for each turnstile reporting, finds the rate of passengers per hour
date_df['hours_elapsed'] = (date_df['turnstile_hours']/date_df['num_turnstiles'])
date_df['passengers_per_hour'] = date_df.total_passengers/date_df.hours_elapsed

In [28]:
date_df.head()

,station,date_time,total_passengers,turnstile_hours,num_turnstiles,hours_elapsed,passengers_per_hour
0,1 AV,2017-05-01 04:00:00,690.000,36.000,9,4.000,172.500
1,1 AV,2017-05-01 08:00:00,5100.000,40.000,10,4.000,1275.000
2,1 AV,2017-05-01 12:00:00,10538.000,40.000,10,4.000,2634.500
3,1 AV,2017-05-01 16:00:00,8687.000,40.000,10,4.000,2171.750
4,1 AV,2017-05-01 20:00:00,12671.000,40.000,10,4.000,3167.750


In [29]:
#pickle the data frame
date_df.to_pickle('date_df.pkl')

In [30]:
#load data from pickle
date_df = pd.read_pickle('date_df.pkl')

In [31]:
'''
Create aggregation for new data frame.
Sum total passengers across turnstile within a station
Sum turnstile recordings within each time segment. This is used as a 'check' column.
Sum hours elapsed with each time segment, This is used as a 'check' against the turnstile recordings

'''

aggregations = {
    'total_passengers': 'sum',
    'num_turnstiles': 'sum',
    'date_time': 'count',
    'hours_elapsed': 'sum'
}

In [32]:
#create dateframe off total passangers per station
station_df = date_df.groupby(('station')).agg(aggregations).reset_index()

#create column giving the average rate of passangers at a station for all hours elapsed
station_df['passengers_per_hour'] = station_df.total_passengers/station_df.hours_elapsed
station_df.rename({'hours_elapsed': 'station_hours', 'num_turnstiles': 'num_observations', 'date_time': 'date_time_count'}, axis = 1, inplace = True)

#create a column tracking the number of recordings for a station divided by hours elapsed in dataset
station_df['num_turnstiles'] = station_df.num_observations/station_df.date_time_count

station_df.head()

,station,total_passengers,num_observations,date_time_count,station_hours,passengers_per_hour,num_turnstiles
0,1 AV,7391182.000,10727,1104,4385.412,1685.402,9.716
1,103 ST,1485584.000,3242,1092,4351.403,341.403,2.969
2,104 ST,591491.000,5656,1107,4457.275,132.702,5.109
3,111 ST,1246868.000,9494,2152,8576.317,145.385,4.412
4,116 ST,1599273.000,4966,1093,4348.625,367.765,4.543


In [33]:
#pickle the data frame
station_df.to_pickle('station_df.pkl')

In [34]:
station_df = pd.read_pickle('station_df.pkl')